# Load data from Postgres to Delta Lake

This notebook shows you how to import data from JDBC Postgres databases into a Delta Lake table using Python.

## Step 1: Connection information

First define some variables to programmatically create these connections.

Replace all the variables in angle brackets `<>` below with the corresponding information.

In [0]:
driver = "org.postgresql.Driver"

database_host = "psql-mock-database-cloud.postgres.database.azure.com"
database_port = "5432" # update if you use a non-default port
database_name = "ecom1692822932465doktuhnaimdbuunj" # eg. postgres

table = "<table-name>" # if your table is in a non-default schema, set as <schema>.<table-name> 
user = "jlfironqhsjytbunvqmfeakr@psql-mock-database-cloud"
password = "hqfrolfvwysvglkclmcolmph"

url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"

print(url)

jdbc:postgresql://psql-mock-database-cloud.postgres.database.azure.com:5432/ecom1692822932465doktuhnaimdbuunj


The full URL printed out above should look something like:

```
jdbc:postgresql://localhost:5432/my_database
```

### Check connectivity

Depending on security settings for your Postgres database and Databricks workspace, you may not have the proper ports open to connect.

Replace `<database-host-url>` with the universal locator for your Postgres implementation. If you are using a non-default port, also update the 5432.

Run the cell below to confirm Databricks can reach your Postgres database.

In [0]:
%sh
nc -vz "<database-host-url>" 5432

nc: getaddrinfo for host "<database-host-url>" port 5432: Name or service not known


## Step 2: Reading the data

Now that you've specified the file metadata, you can create a DataFrame. Use an *option* to infer the data schema from the file. You can also explicitly set this to a particular schema if you have one already.

First, create a DataFrame in Python, referencing the variables defined above.

In [0]:
remote_table = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", table)
    .option("user", user)
    .option("password", password)
    .load()
)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3848709209871904>, line 8
      1 remote_table = (spark.read
      2     .format("jdbc")
      3     .option("driver", driver)
      4     .option("url", url)
      5     .option("dbtable", table)
      6     .option("user", user)
      7     .option("password", password)
----> 8     .load()
      9 )

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:314, in DataFrameReader.load(self, path, format, schema, **options)
    312     return self._df(self.

You can view the results of this remote table query.

In [0]:
display(remote_table)

## Step 3: Create a Delta table

The DataFrame defined and displayed above is a temporary connection to the remote database.

To ensure that this data can be accessed by relevant users throughout your workspace, save it as a Delta Lake table using the code below.

In [0]:
target_table_name = "<target-schema>.<target-table-name>"
remote_table.write.mode("overwrite").saveAsTable(target_table_name)

This table will persist across cluster sessions, notebooks, and personas throughout your organization.

The code below demonstrates querying this data with Python and SQL.

In [0]:
display(spark.table(target_table_name))

In [0]:
%sql
SELECT * FROM <target-schema>.<target-table-name>